In [1]:
from Bio import SeqIO
import pandas as pd

#Analisar o arquivo fasta
fasta_sequences = SeqIO.parse(open('proteins.fasta'),'fasta')

# Criar um dataframe do pandas vazio
fasta_df = pd.DataFrame(columns=['ID', 'Sequence'])

# Percorrer as sequências fasta e adicionando-as ao dataframe do pandas
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    chrom,start,end = fasta.description.split(' ')[1].split(':')
    fasta_df = fasta_df.append({'ID': name, 'Sequence': sequence, 'chrom': chrom, 'start': start, 'end': end}, ignore_index=True)

print(fasta_df)



             ID                                           Sequence  \
0     DJ41_3816  MTTLKITGMTCDSCAAHVKEALEKVPGVQSALVSYPKGTAQLAIEA...   
1     DJ41_3817  MGLITRIAGKTGALGSVVSAMGCAACFPAIASFGAAIGLGFLSQYE...   
2     DJ41_3818  MKKLFASLALAAFVAPVFAATQTVTLSVPGMTCASCPITVKHALSK...   
3     DJ41_3819  MSEPQNGRGALFAGGLAAILASACCLGPLVLIALGFSGAWIGNLTV...   
4     DJ41_3820  MAGAHEFRQHGFHARQVGHLLAHVLELVFGQAAGLLAVGAIVEPQQ...   
...         ...                                                ...   
3745  DJ41_3792  MNSVHEIIEKIHNEWEIEPKKAIQRGMECPFPLQCSLNLKSKIYPQ...   
3746  DJ41_3793  METFAEPVFSNDLLAKAESGDTSAQLELAEIYLYGHGVDSDENQAE...   
3747  DJ41_3794  MSNNKPLKNSTKLLVNLDKFIFLVNAADSLEEIEIIRDLCCEYFSH...   
3748  DJ41_3795  MAKSLQELLASRSPESQARIQKMADELLLESQLHLIREELEISQKE...   
3749  DJ41_3796  MWTVITTDLFNEWLEQQDEATQEKVLAALVVLQQQGPSLGRPLVDT...   

           chrom    end  start  
0     KL810966.1   1844    159  
1     KL810966.1   2308   1883  
2     KL810966.1   2611   2336  
3     KL810966.1   2975   2

In [2]:
fasta_df

,ID,Sequence,chrom,end,start
0,DJ41_3816,MTTLKITGMTCDSCAAHVKEALEKVPGVQSALVSYPKGTAQLAIEA...,KL810966.1,1844,159
1,DJ41_3817,MGLITRIAGKTGALGSVVSAMGCAACFPAIASFGAAIGLGFLSQYE...,KL810966.1,2308,1883
2,DJ41_3818,MKKLFASLALAAFVAPVFAATQTVTLSVPGMTCASCPITVKHALSK...,KL810966.1,2611,2336
3,DJ41_3819,MSEPQNGRGALFAGGLAAILASACCLGPLVLIALGFSGAWIGNLTV...,KL810966.1,2975,2625
4,DJ41_3820,MAGAHEFRQHGFHARQVGHLLAHVLELVFGQAAGLLAVGAIVEPQQ...,KL810966.1,3400,3038
...,...,...,...,...,...
3745,DJ41_3792,MNSVHEIIEKIHNEWEIEPKKAIQRGMECPFPLQCSLNLKSKIYPQ...,KL810967.1,14523,14011
3746,DJ41_3793,METFAEPVFSNDLLAKAESGDTSAQLELAEIYLYGHGVDSDENQAE...,KL810967.1,15080,14562
3747,DJ41_3794,MSNNKPLKNSTKLLVNLDKFIFLVNAADSLEEIEIIRDLCCEYFSH...,KL810967.1,15464,15255
3748,DJ41_3795,MAKSLQELLASRSPESQARIQKMADELLLESQLHLIREELEISQKE...,KL810967.1,15759,15457


In [3]:
fasta_df.iloc[4]

ID                                                  DJ41_3820
Sequence    MAGAHEFRQHGFHARQVGHLLAHVLELVFGQAAGLLAVGAIVEPQQ...
chrom                                              KL810966.1
end                                                      3400
start                                                    3038
Name: 4, dtype: object

In [4]:
#juntando no fasta_df o resultado obtido pelo abricate:
df_abricate = pd.read_csv('../tests/abricate.tab',sep='\t')

abricate_hits = []

for _,row in fasta_df.iterrows():
    best_hit = None
    best_ioc = 0
    for _,abricate_row in df_abricate.iterrows():
        if row.chrom.split('.')[0] == abricate_row.SEQUENCE:
            row_interval = set(range(int(row.start)-1, int(row.end)))
            row_abricate_interval = set(range(abricate_row.START-1, abricate_row.END))
            intersection = len(row_interval.intersection(row_abricate_interval))
            union = len(row_interval.union(row_abricate_interval))
            ioc = intersection / union
            if ioc > best_ioc:
                best_hit = abricate_row.GENE
                best_ioc = ioc
    abricate_hits.append(best_hit)
fasta_df['abricate_genes'] = abricate_hits
fasta_df

,ID,Sequence,chrom,end,start,abricate_hit
0,DJ41_3816,MTTLKITGMTCDSCAAHVKEALEKVPGVQSALVSYPKGTAQLAIEA...,KL810966.1,1844,159,None
1,DJ41_3817,MGLITRIAGKTGALGSVVSAMGCAACFPAIASFGAAIGLGFLSQYE...,KL810966.1,2308,1883,None
2,DJ41_3818,MKKLFASLALAAFVAPVFAATQTVTLSVPGMTCASCPITVKHALSK...,KL810966.1,2611,2336,None
3,DJ41_3819,MSEPQNGRGALFAGGLAAILASACCLGPLVLIALGFSGAWIGNLTV...,KL810966.1,2975,2625,None
4,DJ41_3820,MAGAHEFRQHGFHARQVGHLLAHVLELVFGQAAGLLAVGAIVEPQQ...,KL810966.1,3400,3038,None
...,...,...,...,...,...,...
3745,DJ41_3792,MNSVHEIIEKIHNEWEIEPKKAIQRGMECPFPLQCSLNLKSKIYPQ...,KL810967.1,14523,14011,None
3746,DJ41_3793,METFAEPVFSNDLLAKAESGDTSAQLELAEIYLYGHGVDSDENQAE...,KL810967.1,15080,14562,None
3747,DJ41_3794,MSNNKPLKNSTKLLVNLDKFIFLVNAADSLEEIEIIRDLCCEYFSH...,KL810967.1,15464,15255,None
3748,DJ41_3795,MAKSLQELLASRSPESQARIQKMADELLLESQLHLIREELEISQKE...,KL810967.1,15759,15457,None


In [5]:
df_abricate

,#FILE,SEQUENCE,START,END,STRAND,GENE,COVERAGE,COVERAGE_MAP,GAPS,%COVERAGE,%IDENTITY,DATABASE,ACCESSION,PRODUCT,RESISTANCE
0,tests/genomes/acineto.baum.plasm.gb,KL810966,761166,762512,+,abeM,1-1347/1347,===============,0/0,100.00,100.00,card,AB204810:186-1533,AbeM is an multidrug efflux pump found in Acin...,acridine_dye;fluoroquinolone;triclosan
1,tests/genomes/acineto.baum.plasm.gb,KL810966,1133625,1134440,+,sul2,1-816/816,===============,0/0,100.00,100.00,card,AY055428.1:21084-20268,Sul2 is a sulfonamide resistant dihydropteroat...,sulfonamide
2,tests/genomes/acineto.baum.plasm.gb,KL810966,1923193,1924647,-,adeK,1-1455/1455,===============,0/0,100.00,98.83,card,AY769962:5610-7065,AdeK is the outer membrane factor protein in t...,carbapenem;cephalosporin;diaminopyrimidine;flu...
3,tests/genomes/acineto.baum.plasm.gb,KL810966,1924647,1927823,-,adeJ,1-3177/3177,===============,0/0,100.00,99.56,card,AY769962:2434-5611,AdeJ is a RND efflux protein that acts as the ...,carbapenem;cephalosporin;diaminopyrimidine;flu...
4,tests/genomes/acineto.baum.plasm.gb,KL810966,1927836,1929086,-,adeI,1-1251/1251,===============,0/0,100.00,99.68,card,CP001182.2:3283426-3284677,AdeI is the membrane fusion protein of the Ade...,carbapenem;cephalosporin;diaminopyrimidine;flu...
5,tests/genomes/acineto.baum.plasm.gb,KL810966,2309203,2310354,-,ADC-2,1-1152/1152,===============,0/0,100.00,99.65,card,AY177427.1:1141-2293,ADC-2 is a beta-lactamase found in Oligella ur...,cephalosporin
6,tests/genomes/acineto.baum.plasm.gb,KL810966,2424478,2425929,-,adeH,1-1452/1452,===============,0/0,100.00,98.62,card,CT025802.2:0-1452,AdeH is the outer membrane channel protein of ...,fluoroquinolone;tetracycline
7,tests/genomes/acineto.baum.plasm.gb,KL810966,2425939,2429118,-,adeF,1-3180/3180,===============,0/0,100.00,97.67,card,CT025801.2:0-3180,AdeF is the membrane fusion protein of the mul...,fluoroquinolone;tetracycline
8,tests/genomes/acineto.baum.plasm.gb,KL810966,2429125,2430345,-,adeG,1-1221/1221,===============,0/0,100.00,98.36,card,CT025800.2:0-1221,AdeG is the inner membrane transporter of the ...,fluoroquinolone;tetracycline
9,tests/genomes/acineto.baum.plasm.gb,KL810966,2430561,2431574,+,adeL,1-1014/1014,===============,0/0,100.00,99.70,card,KR297239.1:1428-414,AdeL is a regulator of AdeFGH in Acinetobacter...,fluoroquinolone;tetracycline


KeyError: 'query.id'

In [18]:
#o fasta_df tá juntando as inform que cada programa produziu. Abaixo: estamos lendo o resultado do blast contra o bacmet:
#estamos juntando as informações que estavam no fasta_df e criamos 3 colunas novas bacmet compound, bacmet gene name e bacmet id
#passa por cada linha do dataframe fasta_df e busca as linhas com o mesmo id do gene no dataframe dos resultados do bacmet
df_bacmet['ID'] = df_bacmet['query_id'].apply(lambda x: x.split(' ')[0])

bacmet_compound = []
bacmet_gene_name = []
bacmet_id = []

for r, row in fasta_df.iterrows():
    df_query = df_bacmet.query('ID == @row.ID')
    if df_query.shape[0] > 0:
        bacmet_compound.append(df_query.iloc[0].compound)
        bacmet_gene_name.append(df_query.iloc[0].gene_name)
        bacmet_id.append(df_query.iloc[0].bacmet_id)
    else:
        bacmet_compound.append(None)
        bacmet_gene_name.append(None)
        bacmet_id.append(None)

fasta_df['bacmet_compound'] = bacmet_compound
fasta_df['bacmet_gene'] = bacmet_gene_name
fasta_df['bacmet_id'] = bacmet_id

fasta_df[~pd.isna(fasta_df['bacmet_id'])]

,ID,Sequence,chrom,end,start,abricate_hit,bacmet_compound,bacmet_gene,bacmet_id
0,DJ41_3816,MTTLKITGMTCDSCAAHVKEALEKVPGVQSALVSYPKGTAQLAIEA...,KL810966.1,1844,159,None,Mercury (Hg),merA,BAC0652
5,DJ41_3649,MSYYHKKAPKKNLPLVIYGQDAHQVQQVQKNLPKLMILVVGETARA...,KL810966.1,4611,3580,None,Iron (Fe),pmrC,BAC0485
135,DJ41_2666,MTDAQSNVQETVPTTSASDDNMQNKRKKFLGFFALILLIAAILYAI...,KL810966.1,140055,138904,None,Sodium Deoxycholate (SDC) [class: Acid],vceA,BAC0418
136,DJ41_2665,MKTQTPFAELSGGRLLLAAFVIALSNFMVVLDTTIANVSVPHITGN...,KL810966.1,141585,140062,None,Sodium Deoxycholate (SDC) [class: Acid],vceB,BAC0419
238,DJ41_2563,MNSNLPPVSDSKLAANLQAKSTNVHVPTPKFFMPVFLTIIIATLIY...,KL810966.1,252605,250974,None,"Zinc (Zn), Tellurium (Te)",pitA,BAC0312
489,DJ41_1006,MSHKTISSLVAVFGLLVSPWTLAAIKEYHLNINEQQVNVTGKPLKR...,KL810966.1,499519,497585,None,Copper (Cu),pcoA,BAC0303
543,DJ41_1065,MQKHDAFAALRYRDFSIVTINQFCLTLAILIQEIIVAYSLYQITKD...,KL810966.1,561531,560281,None,"Cobalt (Co), Nickel (Ni)",nreB,BAC0285
581,DJ41_1103,MAKYSNINSFNALQTLTVGSSNYQIFSLTQAEKKLGDIAKLPKSLK...,KL810966.1,600870,598114,None,Iron (Fe),acn,BAC0003
600,DJ41_1122,MTQGLLAGKRFLIAGVASKLSIAYGIAQALHREGAELAFTYPNEKL...,KL810966.1,625279,624476,None,Triclosan [class: Phenolic compounds],fabI,BAC0156
730,DJ41_1254,VSNVTSFRSELKQLFHLMLPILITQFAQAGFGLIDTIMAGHLSAAD...,KL810966.1,762512,761166,abeM,"4,6-diamidino-2-phenylindole (DAPI) [class: Di...",abeM,BAC0001


In [22]:
#passar pelo platon
#usar o seqio para carregar as sequencias presentes no genome.plasmid.fasta
#criar uma lista que se chamará plasmid_ids

records = SeqIO.parse('../tests/genome.plasmid.fasta', 'fasta')
plasmid_ids = [record.id for record in records]

fasta_df['platon_plasmids'] = fasta_df['chrom'].map(lambda chrom_id: chrom_id in plasmid_ids)
fasta_df

,ID,Sequence,chrom,end,start,abricate_hit,bacmet_compound,bacmet_gene,bacmet_id,platon_plasmids
0,DJ41_3816,MTTLKITGMTCDSCAAHVKEALEKVPGVQSALVSYPKGTAQLAIEA...,KL810966.1,1844,159,None,Mercury (Hg),merA,BAC0652,False
1,DJ41_3817,MGLITRIAGKTGALGSVVSAMGCAACFPAIASFGAAIGLGFLSQYE...,KL810966.1,2308,1883,None,None,None,None,False
2,DJ41_3818,MKKLFASLALAAFVAPVFAATQTVTLSVPGMTCASCPITVKHALSK...,KL810966.1,2611,2336,None,None,None,None,False
3,DJ41_3819,MSEPQNGRGALFAGGLAAILASACCLGPLVLIALGFSGAWIGNLTV...,KL810966.1,2975,2625,None,None,None,None,False
4,DJ41_3820,MAGAHEFRQHGFHARQVGHLLAHVLELVFGQAAGLLAVGAIVEPQQ...,KL810966.1,3400,3038,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...
3745,DJ41_3792,MNSVHEIIEKIHNEWEIEPKKAIQRGMECPFPLQCSLNLKSKIYPQ...,KL810967.1,14523,14011,None,None,None,None,True
3746,DJ41_3793,METFAEPVFSNDLLAKAESGDTSAQLELAEIYLYGHGVDSDENQAE...,KL810967.1,15080,14562,None,None,None,None,True
3747,DJ41_3794,MSNNKPLKNSTKLLVNLDKFIFLVNAADSLEEIEIIRDLCCEYFSH...,KL810967.1,15464,15255,None,None,None,None,True
3748,DJ41_3795,MAKSLQELLASRSPESQARIQKMADELLLESQLHLIREELEISQKE...,KL810967.1,15759,15457,None,None,None,None,True


In [33]:
#verificar se os nossos genes estão nas ilhas identificadas pelo SSG-LUGIA:
import json

ssg_lugia_islands = []
ssg_lugia_islands_count = 0

with open('../tests/ssg_lugia.json') as reader:
    ssg_lugia = json.loads(reader.read())
    
for chrom in ssg_lugia:
    for island in ssg_lugia[chrom]:
        ssg_lugia_islands_count = ssg_lugia_islands_count + 1
        ssg_lugia_islands.append([ssg_lugia_islands_count, chrom, int(island[0]), int(island[1])])

gene_islands = []
        
for r, row in fasta_df.iterrows():
    row_interval = set(range(int(row.start), int(row.end)))
    for island in ssg_lugia_islands:
        island_interval = set(range(island[2], island[3]))
        intersection = row_interval.intersection(island_interval)
        if row.chrom == island[1].split('.')[0] and len(intersection) > 0:
            gene_islands.append(island[0])
            break
    else:
        gene_islands.append(None)


In [42]:
fasta_df['gene_island'] = gene_islands
fasta_df[fasta_df['platon_plasmids'] == True]

,ID,Sequence,chrom,end,start,abricate_hit,bacmet_compound,bacmet_gene,bacmet_id,platon_plasmids,gene_island
3722,DJ41_3769,MPKLKDIALGIIVAPLLIPIMLIASYQDKKALKKELDECQKEKDQAS*,KL810967.1,377,234,None,None,None,None,True,None
3723,DJ41_3770,MAKLSLSEVSKKFHVDRSTIYRAVRNGRLSRSSDGQFDLAEVIRCF...,KL810967.1,972,397,None,None,None,None,True,None
3724,DJ41_3771,MRELVVKDNALINASYNLDLVEQRLILLAIVEARESGKGINANDPL...,KL810967.1,1915,965,None,None,None,None,True,None
3725,DJ41_3772,MYNNKYIVFLIFIKVLYEKSVVLVYNYERSKLINKINNLFQKMGFL...,KL810967.1,2969,2781,None,None,None,None,True,None
3726,DJ41_3773,MGEKVKTYAVRLDPQVAEFYDQLANSEGIRTSKLFSKILTNDFQSI...,KL810967.1,3497,3126,None,None,None,None,True,None
3727,DJ41_3774,MSDNIIQLSKPCTLCENRQNVQLFAGLMLCESCQENIRLTNPDMFS*,KL810967.1,3637,3497,None,None,None,None,True,None
3728,DJ41_3775,MDKQRVFFCGNYLIEKREKSQIKKNAKAGDVKSMLDLARYYYANKK...,KL810967.1,4105,3746,None,None,None,None,True,None
3729,DJ41_3776,MSNGTNYFKYIGLEFIKFLIPISITFLTACFFSKLFSSKELFEIYV...,KL810967.1,4712,4200,None,None,None,None,True,None
3730,DJ41_3777,MSNSNVVKGTVKWFNETKGFGFIQQESGPDVFAHFSEIASSGFKTL...,KL810967.1,5254,5039,None,None,None,None,True,None
3731,DJ41_3778,LLKVNDLIAKSKNGTEIIVSLIPLHKMQNTRQGLKQIEVGKRVLLE...,KL810967.1,5579,5352,None,None,None,None,True,None


In [34]:
ssg_lugia_islands

[[1, 'KL810966.1', 125501, 175699],
 [2, 'KL810966.1', 321201, 362999],
 [3, 'KL810966.1', 1637501, 1649199],
 [4, 'KL810966.1', 1946601, 1986199],
 [5, 'KL810966.1', 2371501, 2415499],
 [6, 'KL810966.1', 3297701, 3323299],
 [7, 'KL810966.1', 3535501, 3561799],
 [8, 'KL810966.1', 3639201, 3703299]]